Copyright **`(c)`** 2023 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [40]:
from itertools import product
from random import random, choice, randint, shuffle, seed
from functools import reduce
from collections import namedtuple
from queue import PriorityQueue, SimpleQueue, LifoQueue
from scipy import sparse
from copy import copy

import numpy as np

In [41]:
NUM_SET = 5_000
NUM_POINTS = 5_000
DENSITY = .3
COUNT = 0
tabu_list=[]

In [42]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

# Halloween Challenge

Find the best solution with the fewest calls to the fitness functions for:

* `num_points = [100, 1_000, 5_000]`
* `num_sets = num_points`
* `density = [.3, .7]` 

In [43]:
x = make_set_covering_problem(NUM_SET, NUM_POINTS, DENSITY)


In [44]:

def fitness(state): 
    global COUNT
    COUNT = COUNT + 1
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,  #x[i,j] --> it is a matrix 
            [[x[i,j] for j in range(NUM_POINTS)] for i, t in enumerate(state) if t], 
            np.array([False for _ in range(NUM_POINTS)]),
        )
    )
    return valid, -cost

In [45]:
def tweak(state): 
    new_state = copy(state)
    index = randint(0, NUM_POINTS - 1)
    new_state[index] = not new_state[index]
    return new_state

In [48]:
#PROFESSOR VERSION FOR COMPARING THE RESULTS
current_state = [False for _ in range(NUM_SET)] 

for step in range(500):
    new_state = tweak(current_state)
    if fitness(new_state) >= fitness(current_state):
        current_state = new_state


In [ ]:
print(current_state)

In [46]:

current_state = [False for _ in range(NUM_SET)] #set up of current state --> it starts all False
COUNT=0
#This version is updated with the tabu_list. In the tabu_list we insert the states already encountered, so in the future the computation will be faster 
for step in range(500):
    new_state = tweak(current_state)
    if not new_state in tabu_list:
        if fitness(new_state) >= fitness(current_state):
            current_state = new_state
            tabu_list.append(current_state)
        else:  
            #new state will be inserted in the list of the ones that will be excluded
            tabu_list.append(new_state) 
 

In [ ]:
print(COUNT) #COUNT is the number of call to the function fitness
print(fitness(current_state)) #we stamp ncurrent_state to see the final arriving point 